In [14]:
from dash import dcc, html, Input, Output
import dash_bootstrap_components as dbc
from jupyter_dash import JupyterDash
import plotly.express as px
import pandas as pd

# Load the Excel data from your file
data = pd.read_excel("data.xlsx", sheet_name="Top300")
data.head()


,Year,Name,Position,Age,Club Name,State,League Name,Country,Matches Played,Goals,Assists,Points,Avg points
0,1986,Eloy Olaya,Centre-Forward,21,Sporting GijÃ³n,Rome,LaLiga2,Germany,43,11,0,11,3.909091
1,1986,JosÃ© Mari Bakero,Attacking Midfield,23,Real Sociedad,Saint-Étienne,LaLiga,France,39,11,0,11,3.545455
2,1986,Yannick Stopyra,Centre-Forward,25,Toulouse,Bologna,Ligue 1,Germany,38,11,0,11,3.454545
3,1979,Paco Rubio,Attacking Midfield,25,AS Nancy,Lens,Championnat National,Germany,37,11,0,11,3.363636
4,1985,Yannick Stopyra,Centre-Forward,24,Toulouse,Cologne,Ligue 1,Spain,37,11,0,11,3.363636


In [13]:
from dash import dcc, html, Input, Output
import dash_bootstrap_components as dbc
from jupyter_dash import JupyterDash
import plotly.express as px
import pandas as pd

# Load the Excel data from your file
data = pd.read_excel("data.xlsx", sheet_name="Top300")

# Calculate the average number of goals for each position
average_goals_by_position = data.groupby(['Position', 'Country'])['Goals'].mean().reset_index()

# Create the initial bar chart with color based on positions using Plotly Express
fig_bar = px.bar(average_goals_by_position, x='Position', y='Goals', color='Country', title='Average Goals by Position')

# Customize the bar chart
fig_bar.update_xaxes(title_text='Position')
fig_bar.update_yaxes(title_text='Average Goals')

# Create a pie chart with color based on positions using Plotly Express
fig_pie = px.pie(data, names='Position', title='Distribution of Players by Position')

df = pd.read_excel("data.xlsx", sheet_name="Top300")

# Define initial values for dropdown
selected_country = "All"

# Dropdown for selecting a country
country_dropdown = dcc.Dropdown(
    id='country-dropdown',
    options=[{'label': country, 'value': country} for country in data['Country'].unique()] + [{'label': 'All', 'value': 'All'}],
    value=selected_country,
    clearable=False,
    style={'width': '50%', 'margin': 'auto'}  # Center the dropdown
)

# Slider for selecting the range of years
year_slider = dcc.RangeSlider(
    id='year-slider',
    min=df['Year'].min(),
    max=df['Year'].max(),
    step=1,
    marks={str(year): str(year) for year in range(df['Year'].min(), df['Year'].max() + 1)},
    value=[df['Year'].min(), df['Year'].max()],
)

# Dropdown for selecting the metric
metric_dropdown = dcc.Dropdown(
    id='metric-dropdown',
    options=[
        {'label': 'Matches Played', 'value': 'Matches Played'},
        {'label': 'Goals', 'value': 'Goals'},
        {'label': 'Assists', 'value': 'Assists'},
        {'label': 'Points', 'value': 'Points'},
    ],
    value='Matches Played',
)

# Group the data by 'Country', 'State', 'Club Name', and calculate total matches, goals, and assists
location_stats = df.groupby(['Country', 'State', 'Club Name']).agg({
    'Matches Played': 'sum',
    'Goals': 'sum',
    'Assists': 'sum'
}).reset_index()

# Get unique countries and states
countries = df['Country'].unique()
states = df['State'].unique()

# Initialize the JupyterDash app
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Define styles for headings and descriptions
new_heading_style = {
    'backgroundColor': '#007bff',
    'color': 'white',
    'padding': '10px',
    'textAlign': 'center',
    'margin-top': '20px'
}
description_style = {
    'backgroundColor': '#f8f9fa',
    'color': 'black',
    'padding': '10px',
    'textAlign': 'center',
    'margin-top': '10px'
}

# Define the layout of the app using dbc components
app.layout = dbc.Container(
    [
        # Main Title with Background Color
        dbc.Row(
            dbc.Col(html.H1("Football Data Analysis Dashboard", style={'textAlign': 'center', 'backgroundColor': '#f8f9fa', 'padding': '10px'}))
        ),

        # First Visualization - Bar Chart and Pie Chart
        dbc.Row(
            [
                dbc.Col(html.H1("Football Player Analysis")),
                
                dbc.Col(html.Label("Select Country:"), width=2,style={'text-align': 'center'}),
                dbc.Col(country_dropdown, width=4,style={'text-align': 'center','margin': 'auto'}),
                dbc.Col(
                    [
                        dbc.Row(
                            [
                                dbc.Col(dcc.Graph(figure=fig_bar, id='bar-chart'), width=6),
                                dbc.Col(dcc.Graph(figure=fig_pie, id='pie-chart'), width=6),
                            ]
                        ),
                    ],
                    width=12,
                ),
                dbc.Col(html.P("Explore the average goals by position and the distribution of players by position.", style={'text-align': 'center'})),
            ]
        ),

        # Second Visualization - Line Chart and TreeMap Side by Side
        dbc.Row(
            [
                # Line Chart Column
                dbc.Col(
                    [
                        html.H1("Player Performance Over the Years", style={'text-align': 'center'}),
                        html.Label("Select Year Range:"),
                        year_slider,
                        html.Label("Select Metric:"),
                        metric_dropdown,
                        dcc.Graph(id='line-chart'),
                        html.Div(id='graph-description'),
                    ],
                    style={'width': 6, 'margin': 'auto'},
                ),

                # TreeMap Column
                dbc.Col(
                    [
                        html.H1("Interactive TreeMap", style={'text-align': 'center'}),
                        dcc.Dropdown(
                            id='year-dropdown',
                            options=[
                                {'label': year, 'value': year}
                                for year in df['Year'].unique()
                            ],
                            value=df['Year'].min(),
                            style={'width': '50%', 'margin': 'auto'},
                        ),
                        dcc.Graph(id='treemap'),
                        html.Div(
                            [
                                html.P("The TreeMap provides a detailed overview of the distribution of matches played, goals, and assists across countries, states, and clubs. "
                                       "Hover over each section to see more details.")
                            ],
                            style=description_style,
                        ),
                    ],
                    style={'width': 6, 'margin': 'auto'},
                ),
            ]
        ),

        # Third Visualization - Country and State Visualization
        dbc.Row(
            [
                dbc.Col(
                    [
                        html.H1("Country and State Visualization"),
                        dcc.RadioItems(
                            id='country-selector',
                            options=[{'label': country, 'value': country} for country in countries],
                            value=countries[0],
                            labelStyle={'display': 'block'}
                        ),
                        dcc.Dropdown(
                            id='state-dropdown',
                            options=[{'label': state, 'value': state} for state in states],
                            style={'width': '50%'}
                        ),
                        dcc.Graph(id='country-state-visualization'),
                        html.Div(
                            [
                                html.P("Explore the total goals scored by position in a specific country and state. "
                                       "Select the country and state to view the visualization.")
                            ],
                            style=description_style,
                        ),
                    ],
                    width=12,
                ),
            ]
        ),
    ],
    fluid=True,
)

# Description element
description = dbc.Container(
    [
        dbc.Row(
            dbc.Col(
                [
                    html.H3("Analysis Overview"),
                    html.P("Explore the trend in player performance over the years based on various metrics. "
                           "Use the sliders and dropdown to customize the visualization."),
                ],
                width=6,
            ),
        ),
    ],
    fluid=True,
)

# Callbacks for Interactive Visualizations
@app.callback(
    [Output('bar-chart', 'figure'),
     Output('pie-chart', 'figure')],
    [Input('country-dropdown', 'value')]
)
def update_visualizations(selected_country):
    # Filter data based on selected country
    if selected_country != 'All':
        filtered_data_bar = average_goals_by_position[average_goals_by_position['Country'] == selected_country]
        filtered_data_pie = data[data['Country'] == selected_country]
    else:
        filtered_data_bar = average_goals_by_position
        filtered_data_pie = data

    # Update the bar chart
    fig_bar = px.bar(filtered_data_bar, x='Position', y='Goals', color='Country',
                     title=f'Average Goals by Position ({selected_country})')
    fig_bar.update_xaxes(title_text='Position')
    fig_bar.update_yaxes(title_text='Average Goals')

    # Update the pie chart
    fig_pie = px.pie(filtered_data_pie, names='Position', title=f'Distribution of Players by Position ({selected_country})')

    return fig_bar, fig_pie

# Third Visualization Callback
@app.callback(
    [Output('line-chart', 'figure'),
     Output('graph-description', 'children')],
    [Input('year-slider', 'value'),
     Input('metric-dropdown', 'value')]
)
def update_line_chart(year_range, selected_metric):
    # Filter data based on selected year range
    filtered_data = df[(df['Year'] >= year_range[0]) & (df['Year'] <= year_range[1])]

    # Group data by year and calculate the mean for each metric
    yearly_stats = filtered_data.groupby("Year")[["Matches Played", "Goals", "Assists", "Points"]].mean().reset_index()

    # Create an interactive line chart using Plotly Express
    fig_line_chart = px.line(
        yearly_stats,
        x='Year',
        y=selected_metric,
        title=f"Trend in {selected_metric} Over the Years",
    )

    # Description text
    description_text = f"The line chart illustrates the trend in {selected_metric} over the selected years."

    return fig_line_chart, description_text

# Fourth Visualization Callback
@app.callback(
    Output('treemap', 'figure'),
    [Input('year-dropdown', 'value')]
)
def update_treemap(selected_year):
    filtered_data = df[df['Year'] == selected_year]

    fig = px.treemap(
        filtered_data,
        path=['Country', 'State', 'Club Name'],
        values='Matches Played',  # Use 'Matches Played' as the main variable
        color='Assists',  # Color the treemap based on total goals
        hover_data=['Matches Played', 'Goals', 'Assists'],  # Display additional information on hover
        title=f'Detailed TreeMap - Country, State, and Club Name - Year {selected_year}',
    )

    return fig

# Fifth Visualization Callbacks
@app.callback(
    Output('state-dropdown', 'options'),
    Output('state-dropdown', 'value'),
    Input('country-selector', 'value')
)
def update_state_options(selected_country):
    # Filter the dataset based on the selected country
    country_data = df[df['Country'] == selected_country]

    # Get unique states in the selected country
    country_states = country_data['State'].unique()

    # Set default value to the first state in the list
    default_value = country_states[0]

    return [{'label': state, 'value': state} for state in country_states], default_value

# Callback to update the visualization based on the selected country and state
@app.callback(
    Output('country-state-visualization', 'figure'),
    [Input('country-selector', 'value'),
     Input('state-dropdown', 'value')]
)
def update_country_state_visualization(selected_country, selected_state):
    # Filter the dataset based on the selected country and state
    filtered_data = df[(df['Country'] == selected_country) & (df['State'] == selected_state)]

    # Group data by position and calculate the total number of goals
    position_goals = filtered_data.groupby('Position')['Goals'].sum().reset_index()

    # Create a bar chart showing total goals by position
    fig = px.bar(
        position_goals,
        x='Position',
        y='Goals',
        labels={'Position': 'Position', 'Goals': 'Total Goals'},
        title=f'Total Goals by Position in {selected_country} - {selected_state} State',
        color='Position'
    )

    fig.update_layout(xaxis_title='Position', yaxis_title='Total Goals')

    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(mode='external', port=4444)


C:\Users\pegad\anaconda3\Lib\site-packages\dash\dash.py:538: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



Dash app running on http://127.0.0.1:4444/
